## HAADF and Diffraction only
- Haadf
- sample 20 locations - get diffraction
- pca plots of eels on the haadf

In [ ]:
from stemOrchestrator.logging_config import setup_logging

data_folder = "."
out_path = data_folder
setup_logging(out_path=out_path)

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
import Pyro5.api

plot = plt
from typing import Dict

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF")
            port = port or secret.get("port_TF")


if ip is None:
    print("please check path of yaml file containing ip and port info")

else:
    print("your yaml file with ip and port loaded fine")
config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512,  # square
    "out_path": ".",
}

In [ ]:
ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]


microscope = TemMicroscopeClient()
microscope.connect(ip, port=port)  # 7521 on velox  computer
# microscope.connect( port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope, offline=False)
uri = "PYRO:array.server@10.46.217.242:9094"


# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])

In [ ]:
# get haadf from mic

# Get haadf
haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(
    exposure=haadf_exposure, resolution=haadf_resolution
)

HAADF_tiff_to_png(haadf_tiff_name)
haadf = haadf_np_array

W, H = haadf.shape

In [ ]:
# best haadf after normalization
haadf_normalized = haadf

plt.imshow(haadf_normalized)

In [ ]:
# finds positions to sample from
import numpy as np
N = 20
rng = np.random.default_rng(42)
xs = rng.integers(0, W, size=N)
ys = rng.integers(0, H, size=N)
pixel_pos = np.stack([xs, ys], axis=1)

In [ ]:
# plot position sampled
plt.imshow(haadf_normalized)
plt.scatter(xs, ys, s=50, c="r")

In [ ]:
# get edx at those positon and stack them
ceta_exposure = 0.1  # seconds
ceta_resolution = 1024
all_arrays = []  # 1. Create an empty list before the loop
for point in pixel_pos:
    # convert to fractional coordinates
    x_pos = point[0] / W
    y_pos = point[1] / H

    # position beam
    tf_acquisition.move_paused_beam(x_pos, y_pos)

    # Acquire the EDS spectrum
    microscope.optics.blanker.unblank()
    ceta_cp_array, ceta_tiff_name = tf_acquisition.acquire_ceta_or_flucam(
        exposure=ceta_exposure, resolution=ceta_resolution, camera="ceta"
    )    
    microscope.optics.blanker.blank()
    
    # clip the bright spots
    shifted_data = ceta_cp_array
    p99 = np.percentile(shifted_data.ravel(), 99)
    clipped_data = np.clip(shifted_data, 0, p99)
    clipped_data -= clipped_data.min()
    clipped_data /= clipped_data.max()
    norm_data = clipped_data
    # power law 2nd time through
    gamma = 1
    norm_data = norm_data**gamma
    edge_crop = 256
    norm_data = norm_data[edge_crop:-edge_crop, edge_crop:-edge_crop]
    
    plt.imshow(norm_data)
    plt.show()

    #######----> need to create energy axis but lets do later

    # stack the arrays
    all_arrays.append(norm_data)  # 2. Add the new array to the list

spectra = np.stack(all_arrays, axis=0)
print(spectra.shape)

In [ ]:
# normalize spectra of shape (20, 512, 512)
mins = spectra.min(axis=(1, 2), keepdims=True)
ptps = np.ptp(spectra, axis=(1, 2), keepdims=True)
ptps = np.where(ptps == 0, 1.0, ptps)
spectra_norm = (spectra - mins) / ptps


In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# reshape to (20, 512*512)
spectra_flat = spectra_norm.reshape(spectra_norm.shape[0], -1)

k = 3
pca = PCA(n_components=k, random_state=42)
scores = pca.fit_transform(spectra_flat)  # shape (20, k)


In [ ]:
# imshow the pc1 values on the haadf
# -----------------------------
# 5) Plots
# -----------------------------

# a) HAADF overlay with PC1 color
plt.figure(figsize=(6, 6))
plt.imshow(haadf_normalized, cmap="gray")
plt.scatter(xs, ys, c=scores[:, 0], s=50)
plt.title("HAADF + Diffraction PCA Overlay (PC1 color)")
plt.colorbar(label="PC1 score", cmap="magma")
plt.tight_layout()
# plt.savefig("/mnt/data/overlay_plot.png", dpi=160)
plt.show()

In [ ]:
# KMeans on PCA scores
km = KMeans(n_clusters=3, n_init=10, random_state=42)
cluster_labels = km.fit_predict(scores)

In [ ]:
# b) PCA scatter with clusters
plt.figure(figsize=(6, 5))
plt.scatter(scores[:, 0], scores[:, 1], s=8, alpha=0.8, c=cluster_labels)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA Scatter of Diffraction with KMeans Clusters")
plt.tight_layout()
# plt.savefig("/mnt/data/pca_scatter.png", dpi=160)
plt.show()

## SAM + Diffraction

In [ ]:
from stemOrchestrator.logging_config import setup_logging
import os

data_folder = "out-diffraction/"

os.makedirs(data_folder, exist_ok=True)
out_path = data_folder
setup_logging(out_path=out_path)

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging

plot = plt
from typing import Dict

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF")
            port = port or secret.get("port_TF")


if ip is None:
    print("please check path of yaml file containing ip and port info")

else:
    print("your yaml file with ip and port loaded fine")
config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512,  # square
    "out_path": f"{data_folder}",
}

In [ ]:
ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]


microscope = TemMicroscopeClient()
microscope.connect(ip, port=port)  # 7521 on velox  computer
# microscope.connect( port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])

In [ ]:
# get haadf from mic

# Get haadf
haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(
    exposure=haadf_exposure, resolution=haadf_resolution, folder_path=out_path
)

HAADF_tiff_to_png(out_path + haadf_tiff_name)
haadf = haadf_np_array

W, H = haadf.shape

In [ ]:
# best haadf after normalization
haadf_normalized = haadf_np_array

plt.imshow(haadf_normalized)

In [ ]:
## get positons from sam

########SAM part ********************************************************************************************************
# -----> takes quite some time- 3 minutes-- to load --
from stemOrchestrator.MLlayer.MLlayerSAM import (
    setup_device,
    download_sam_model,
    initialize_sam_model,
    preprocess_image,
    generate_and_save_masks,
    create_normalized_particle_positions,
    display_image_with_masks,
    display_image_with_labels,
    extract_mask_contours,
    generate_mask_colors,
    visualize_masks_with_boundaries,
    extract_particle_data,
    print_boundary_points_info,
    plot_centroids,
    sample_particle_positions,
    plot_sampled_positions,
    create_visualization_with_masks,
)
import pickle
import numpy as np
from typing import List, Dict, Union


def run_sam(image_data: np.ndarray, path_folder: str) -> Union[List, Dict]:
    """Main function to run SAM segmentation pipeline."""
    device = setup_device()

    model_type = "vit_b"  # Options: 'vit_b', 'vit_l', 'vit_h'
    checkpoint_url = (
        "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
    )
    checkpoint_path = "sam_vit_b_01ec64.pth"
    download_sam_model(model_type, checkpoint_url, checkpoint_path)
    sam, mask_generator = initialize_sam_model(model_type, checkpoint_path, device)
    img_np = preprocess_image(image_data)

    plt.figure(figsize=(8, 8))
    plt.imshow(img_np)
    plt.title("Original Image")
    plt.axis("off")
    plt.show()

    # Generate and visualize masks
    masks_path = f"{path_folder}/masks_Au_online.pkl"
    masks = generate_and_save_masks(mask_generator, img_np, masks_path)
    visual_image, centroids = create_visualization_with_masks(img_np, masks)
    display_image_with_masks(visual_image, "Image with Segmentation Masks")
    display_image_with_labels(
        visual_image, centroids, "Image with Segmentation Masks and Labels"
    )

    mask_contours = extract_mask_contours(masks)
    mask_colors = generate_mask_colors(len(masks))
    boundaries_path = (
        f"{path_folder}/Segmentation Masks with Boundaries and Centroids.png"
    )
    visualize_masks_with_boundaries(
        visual_image, centroids, mask_contours, mask_colors, boundaries_path
    )
    particles = extract_particle_data(masks)
    # Save particle data
    # with open(f'{path_folder}/particles.pkl', 'wb') as f:
    #     pickle.dump(particles, f)

    print_boundary_points_info(particles)
    centroids_array = np.array(centroids)
    plot_centroids(centroids_array, img_np)
    positions_sampled = sample_particle_positions(particles, img_np)
    plot_sampled_positions(positions_sampled, img_np, len(centroids))
    each_particle_position = create_normalized_particle_positions(
        particles, img_np.shape[:2]
    )
    # with open(f'{path_folder}/sampled_boundary_pts_particles.pkl', 'wb') as f: # Save normalized particle positions
    #     pickle.dump(each_particle_position, f)

    all_particle_keys = each_particle_position.keys()

    print("Processing complete!")
    return all_particle_keys, each_particle_position


##########****************************************************************************************************************************

In [ ]:
## run the segmentaiotn on haadf to get particles
from datetime import datetime

print(datetime.now().strftime("%Y%m%d_%H%M%S"))
all_particle_keys, each_particle_position = run_sam(
    haadf_np_array, out_path
)  ############ haadf normalized doesnt work here --- weird
print(datetime.now().strftime("%Y%m%d_%H%M%S"))

In [ ]:
# finds positions to sample from
# N = 20
rng = np.random.default_rng(42)
# xs = rng.integers(0, W, size=N)
# ys = rng.integers(0, H, size=N)
# pixel_pos = np.stack([xs, ys], axis=1)

centroids = np.array([v["centroid"] for v in each_particle_position.values()])

# Sample N random centroids (without replacement if fewer than N)
N = len(centroids)
rng = np.random.default_rng(42)
idx = rng.choice(len(centroids), size=min(N, len(centroids)), replace=False)
sampled = centroids[idx]

# Separate xs and ys (keeping same data types as your original code)
xs = sampled[:, 0] * W
ys = sampled[:, 1] * H
pixel_pos = np.stack([xs, ys], axis=1)

print("xs:", xs)
print("ys:", ys)
print("pixel_pos:\n", pixel_pos)

np.save(f"{out_path}pixel_pos.npy", pixel_pos)

In [ ]:
# plot position sampled
plt.imshow(haadf_normalized)
plt.scatter(xs, ys, s=50, c="r")
# want to save this? - nope

In [ ]:
# get edx at those positon and stack them
ceta_exposure = 0.1  # seconds
ceta_resolution = 1024
all_arrays = []  # 1. Create an empty list before the loop
for point in pixel_pos:
    # convert to fractional coordinates
    x_pos = point[0] / W
    y_pos = point[1] / H

    # position beam
    tf_acquisition.move_paused_beam(x_pos, y_pos)

    # Acquire the EDS spectrum
    microscope.optics.blanker.unblank()
    ceta_cp_array, ceta_tiff_name = tf_acquisition.acquire_ceta_or_flucam(
        exposure=ceta_exposure, resolution=ceta_resolution, camera="ceta", folder_path=out_path
    )    
    microscope.optics.blanker.blank()
    
    # clip the bright spots
    shifted_data = ceta_cp_array
    p99 = np.percentile(shifted_data.ravel(), 99)
    clipped_data = np.clip(shifted_data, 0, p99)
    clipped_data -= clipped_data.min()
    clipped_data /= clipped_data.max()
    norm_data = clipped_data
    # power law 2nd time through
    gamma = 1
    norm_data = norm_data**gamma
    edge_crop = 256
    norm_data = norm_data[edge_crop:-edge_crop, edge_crop:-edge_crop]
    
    plt.imshow(norm_data)
    plt.show()

    #######----> need to create energy axis but lets do later

    # stack the arrays
    all_arrays.append(norm_data)  # 2. Add the new array to the list

spectra = np.stack(all_arrays, axis=0)
print(spectra.shape)

In [ ]:
# normalize spectra of shape (20, 512, 512)
mins = spectra.min(axis=(1, 2), keepdims=True)
ptps = np.ptp(spectra, axis=(1, 2), keepdims=True)
ptps = np.where(ptps == 0, 1.0, ptps)
spectra_norm = (spectra - mins) / ptps


In [ ]:
# save raw spectra
np.save(f"{out_path}raw_spectra.npy", spectra)
np.save(f"{out_path}normalized_spectra.npy",spectra_norm)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# reshape to (20, 512*512)
spectra_flat = spectra_norm.reshape(spectra_norm.shape[0], -1)

k = 3
pca = PCA(n_components=k, random_state=42)
scores = pca.fit_transform(spectra_flat)  # shape (20, k)


In [ ]:
# imshow the pc1 values on the haadf
# -----------------------------
# 5) Plots
# -----------------------------

# a) HAADF overlay with PC1 color
plt.figure(figsize=(6, 6))
plt.imshow(haadf_normalized, cmap="gray")
plt.scatter(xs, ys, c=scores[:, 0], s=50)
plt.title("HAADF + Diffraction PCA Overlay (PC1 color)")
plt.colorbar(label="PC1 score", cmap="magma")
plt.tight_layout()
# plt.savefig("/mnt/data/overlay_plot.png", dpi=160)
plt.show()

In [ ]:
# KMeans on PCA scores
km = KMeans(n_clusters=3, n_init=10, random_state=42)
cluster_labels = km.fit_predict(scores)

In [ ]:
# b) PCA scatter with clusters
plt.figure(figsize=(6, 5))
plt.scatter(scores[:, 0], scores[:, 1], s=8, alpha=0.8, c=cluster_labels)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA Scatter of Diffraction with KMeans Clusters")
plt.tight_layout()
# plt.savefig("/mnt/data/pca_scatter.png", dpi=160)
plt.show()